In [ ]:
!pip install qiskit
!pip install qiskit_machine_learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.8/97.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 16.1 MB/s eta 0:00:00
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512549 sha256=40554898896a88f3752d64ff8c0c868a339572103604372f9f97d66079a382e5
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d5

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit_machine_learning.connectors import TorchConnector
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.primitives import Estimator
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Load the data
def load_data(path):
    data = pd.read_csv(path)
    return data

dataset_path = '/content/drive/MyDrive/Classroom/MiniProject_BlackHoleMassEstimation/typeII_AGN_metadata.csv'
df = load_data(dataset_path)

In [ ]:
# Handling missing values by replacing with column mean
df.fillna(df.mean(), inplace=True)

# Target and Features
target_column = 'log_bh_mass'
feature_columns = [
    'h_beta_flux', 'h_beta_flux_err', 'oiii_5007_flux', 'oiii_5007_flux_err',
    'h_alpha_flux', 'h_alpha_flux_err', 'nii_6584_flux', 'nii_6584_flux_err',
    'log_stellar_sigma', 'psfMag_u', 'psfMag_g', 'psfMag_r', 'psfMag_i',
    'psfMag_z', 'psfMagErr_u', 'psfMagErr_g', 'psfMagErr_r', 'psfMagErr_i',
    'psfMagErr_z', 'mendel_logM_p50', 'mendel_logM_p16', 'mendel_logM_p84',
    'mendel_logMt_p50', 'mendel_logMt_p16', 'mendel_logMt_p84',
    'mendel_logMb_p50', 'mendel_logMb_p16', 'mendel_logMb_p84',
    'mendel_logMd_p50', 'mendel_logMd_p16', 'mendel_logMd_p84',
    'simard_b_t_g', 'simard_e_b_t_g', 'simard_b_t_r', 'simard_e_b_t_r',
    'simard_Rhlg', 'simard_Rhlr', 'simard_Rchl_g', 'simard_Rchl_r',
    'simard_Re', 'simard_e_Re', 'simard_e', 'simard_e_e', 'simard_nb',
    'simard_e_nb', 'simard_PpS', 'simard_Pn4'
]

X = df[feature_columns]
y = df[target_column]

In [ ]:
# Normalize the features using Min-Max Scaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA to reduce the dimensionality to 4 components
pca = PCA(n_components=4)
X_pca = pca.fit_transform(X_scaled)

# Convert the reduced dataset to torch tensors
X_pca_tensor = torch.tensor(X_pca, dtype=torch.float32)

# Normalize the target variable using Min-Max Scaler
y_scaler = MinMaxScaler()
y_scaled = y.values.reshape(-1, 1)  # Reshape for scaler
y_scaled = y_scaler.fit_transform(y_scaled)
y_tensor = torch.tensor(y_scaled, dtype=torch.float32)

In [ ]:
# Update the quantum circuit to work with 4 qubits
num_qubits = X_pca.shape[1]  # This should be 4 now
qc = QuantumCircuit(num_qubits)
params = [Parameter(f'theta_{i}') for i in range(num_qubits)]

# Simple parametric circuit with the reduced number of qubits
for i in range(num_qubits):
    qc.rx(params[i], i)

# Define the observable for measurement with the new number of qubits
observable = SparsePauliOp("Z" * num_qubits, 1)

In [ ]:
print(observable)

SparsePauliOp(['ZZZZ'],
              coeffs=[1.+0.j])


In [ ]:
qnn = EstimatorQNN(circuit=qc, observables=observable, input_params=params, estimator=None, input_gradients=True)

qnn_torch = TorchConnector(qnn)

class QNNRegressor(nn.Module):
    def __init__(self, qnn):
        super(QNNRegressor, self).__init__()

        # Classical layers
        self.fc1 = nn.Linear(4, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 16)

        # Quantum Neural Network
        self.qnn = qnn

        # Output layer for combining the classical and quantum outputs
        self.fc4 = nn.Linear(16 + 1, 1)  # Concatenate classical and QNN outputs

    def forward(self, x):
        # Apply classical layers
        x_classical = torch.relu(self.fc1(x))
        x_classical = torch.relu(self.fc2(x_classical))
        x_classical = torch.relu(self.fc3(x_classical))

        # Apply QNN
        x_qnn = self.qnn(x)

        # Concatenate classical and QNN outputs
        x_combined = torch.cat((x_classical, x_qnn), dim=1)

        # Apply final output layer
        return self.fc4(x_combined)

model = QNNRegressor(qnn_torch)

In [ ]:
# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Lower learning rate\

last_loss = 0

# Train the Model
epochs = 100  # Increase number of epochs
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    # Forward pass with the reduced feature tensor
    outputs = model(X_pca_tensor)

    # Compute the loss
    loss = criterion(outputs, y_tensor)

    # Backward pass and optimization
    loss.backward()
    optimizer.step()

    # Store the last loss
    last_loss = loss.item()

    # Print loss for monitoring
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')



Epoch 1/100, Loss: 0.0710
Epoch 2/100, Loss: 0.0671
Epoch 3/100, Loss: 0.0635
Epoch 4/100, Loss: 0.0600
Epoch 5/100, Loss: 0.0565
Epoch 6/100, Loss: 0.0532
Epoch 7/100, Loss: 0.0500
Epoch 8/100, Loss: 0.0468
Epoch 9/100, Loss: 0.0437
Epoch 10/100, Loss: 0.0406
Epoch 11/100, Loss: 0.0377
Epoch 12/100, Loss: 0.0349
Epoch 13/100, Loss: 0.0322
Epoch 14/100, Loss: 0.0296
Epoch 15/100, Loss: 0.0272
Epoch 16/100, Loss: 0.0250
Epoch 17/100, Loss: 0.0230
Epoch 18/100, Loss: 0.0213
Epoch 19/100, Loss: 0.0198
Epoch 20/100, Loss: 0.0186
Epoch 21/100, Loss: 0.0177
Epoch 22/100, Loss: 0.0171
Epoch 23/100, Loss: 0.0167
Epoch 24/100, Loss: 0.0165
Epoch 25/100, Loss: 0.0165
Epoch 26/100, Loss: 0.0166
Epoch 27/100, Loss: 0.0168
Epoch 28/100, Loss: 0.0169
Epoch 29/100, Loss: 0.0170
Epoch 30/100, Loss: 0.0170
Epoch 31/100, Loss: 0.0169
Epoch 32/100, Loss: 0.0167
Epoch 33/100, Loss: 0.0165
Epoch 34/100, Loss: 0.0162
Epoch 35/100, Loss: 0.0158
Epoch 36/100, Loss: 0.0154
Epoch 37/100, Loss: 0.0151
Epoch 38/1

In [ ]:
# Test the Model and Compute Metrics
model.eval()
with torch.no_grad():
    # Use the PCA-reduced tensor for predictions
    predictions = model(X_pca_tensor).numpy().flatten()

# Metrics: MSE
y_range = y.max() - y.min()

mse = last_loss
accuracy = (1-last_loss/y_range)*100

print ("MSE is ", mse)
print ("Accuracy by MSE is ", accuracy),

MSE is  0.012427426874637604
Accuracy by MSE is  99.75353307288032


(None,)